In [12]:
import sys
sys.path.append('./code')
import torch
import cv2
import torch.nn as nn
import numpy as np
import argparse
from vgg_face import *
import matplotlib.pyplot as plt
from facegenDataset import *
from FAUDataset import *
plt.switch_backend('agg')

def set_parameter_requires_grad(model, feature_extracting):
    for param in model.parameters():
        param.requires_grad = False if feature_extracting else True

def test_model(model, crop_frame, device):
    model.eval()
    with torch.no_grad():
        crop_frame = cv2.cvtColor(crop_frame, cv2.COLOR_BGR2RGB)
        crop_frame = cv2.resize(crop_frame, (224, 224))
        crop_frame = crop_frame / 255.0
        crop_frame = np.transpose(crop_frame, (2, 0, 1))
        crop_frame = np.expand_dims(crop_frame, axis=0)  # add batch dimension
        crop_frame = torch.from_numpy(crop_frame).float().to(device)
        output = model(crop_frame)
        output = output * torch.FloatTensor([16] + [5]*9).to(device)
    return output

num_classes = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
model = VGG_16().to(device)
set_parameter_requires_grad(model, feature_extracting=False)
num_ftrs = model.fc8.in_features
model.fc8 = nn.Linear(num_ftrs, num_classes).to(device)
model.load_state_dict(torch.load('/Volumes/Yuan-T7/FAU_models/checkpoint_epoch_init.pth', map_location=device))

crop_frame = cv2.imread('/Volumes/Yuan-T7/Datasets/facegen/images/African/a/4a_4.10.png')
output = test_model(model, crop_frame, device)
output = output.flatten()
print(output)

tensor([0.9452, 0.0779, 0.3747, 0.1388, 0.0467, 0.2784, 0.0078, 0.1205, 0.0840,
        0.0761], device='mps:0')


In [8]:
output_text = '|PSPI {:.2f} |au4 {:.2f} |au6 {:.2f} |au7 {:.2f} |au10 {:.2f} |au12 {:.2f} |au20 {:.2f} |au25 {:.2f} |au26 {:.2f} |au43 {:.2f}|'.format(output[0].item(), output[1].item(), output[2].item(), output[3].item(), output[4].item(), output[5].item(), output[6].item(), output[7].item(), output[8].item(), output[9].item())

RuntimeError: a Tensor with 10 elements cannot be converted to Scalar